In [32]:
import json
import numpy as np
import pandas as pd
from inline_sql import sql
import matplotlib.pyplot as plt

### Importacion de tablas

In [33]:
pozos = pd.read_csv("pozos-con-geo-info.csv")
produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")

/var/folders/r_/x2vzgchn7xg93cptq09838d91bmbgx/T/ipykernel_41544/1715450679.py:2: DtypeWarning: Columns (16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  produccion = pd.read_csv("produccin-de-pozos-de-gas-y-petrleo-2024.csv")


In [34]:
pozos.info(), produccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84332 entries, 0 to 84331
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               84332 non-null  int64  
 1   sigla                    84332 non-null  object 
 2   idpozo                   84332 non-null  int64  
 3   area                     84332 non-null  object 
 4   cod_area                 84332 non-null  object 
 5   empresa                  83435 non-null  object 
 6   yacimiento               84332 non-null  object 
 7   cod_yacimiento           84332 non-null  object 
 8   formacion                81517 non-null  object 
 9   cuenca                   84332 non-null  object 
 10  provincia                84332 non-null  object 
 11  cota                     84332 non-null  float64
 12  profundidad              84332 non-null  float64
 13  clasificacion            84332 non-null  object 
 14  subclasificacion      

(None, None)

### Rellado con Estado Nacional en provincia blanks

In [35]:
pozos['provincia_nombre'] = pozos['provincia_nombre'].fillna('Estado Nacional')
pozos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84332 entries, 0 to 84331
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               84332 non-null  int64  
 1   sigla                    84332 non-null  object 
 2   idpozo                   84332 non-null  int64  
 3   area                     84332 non-null  object 
 4   cod_area                 84332 non-null  object 
 5   empresa                  83435 non-null  object 
 6   yacimiento               84332 non-null  object 
 7   cod_yacimiento           84332 non-null  object 
 8   formacion                81517 non-null  object 
 9   cuenca                   84332 non-null  object 
 10  provincia                84332 non-null  object 
 11  cota                     84332 non-null  float64
 12  profundidad              84332 non-null  float64
 13  clasificacion            84332 non-null  object 
 14  subclasificacion      

### Limpieza pozos no productivos

Tomo los idpozos que tuvieron algo de producción en algún mes del año

In [5]:
produccion['index'] = produccion.index
produccion_nonula = sql^ """
                         SELECT *
                         FROM produccion AS p1
                         WHERE EXISTS ( SELECT p2.idpozo, SUM(p2.prod_pet) AS prod_pet_tot, SUM(p2.prod_gas) AS prod_gas_tot
                                        FROM produccion AS p2
                                        WHERE p1.idpozo = p2.idpozo
                                        GROUP BY p2.idpozo
                                        HAVING prod_pet_tot != 0 OR prod_gas_tot != 0
                                      )
                         """
produccion_nonula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274497 entries, 0 to 274496
Data columns (total 39 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idempresa               274497 non-null  object 
 1   anio                    274497 non-null  int64  
 2   mes                     274497 non-null  int64  
 3   idpozo                  274497 non-null  int64  
 4   prod_pet                274497 non-null  float64
 5   prod_gas                274497 non-null  float64
 6   prod_agua               274497 non-null  float64
 7   iny_agua                274497 non-null  float64
 8   iny_gas                 274497 non-null  float64
 9   iny_co2                 274497 non-null  float64
 10  iny_otro                274497 non-null  float64
 11  tef                     274497 non-null  float64
 12  vida_util               90 non-null      float64
 13  tipoextraccion          274475 non-null  object 
 14  tipoestado          

In [56]:
# Solo para celes que no me anda el inline_sql jaja

# produccion['index'] = produccion.index
# # Calcula las sumas de prod_pet y prod_gas agrupadas por idpozo
# sums = produccion.groupby('idpozo')[['prod_pet', 'prod_gas']].sum()

# # Filtra idpozo donde al menos una columna tenga suma distinta de 0
# non_null_pozo_ids = sums[(sums['prod_pet'] != 0) | (sums['prod_gas'] != 0)].index

# # Filtra el DataFrame original
# produccion_nonula = produccion[produccion['idpozo'].isin(non_null_pozo_ids)]

# # Muestra información del resultado
# produccion_nonula.info()

Filtro en la tabla de pozos los que no tuvieron nada de producción

In [37]:
pozos = pozos[pozos['idpozo'].isin(produccion_nonula['idpozo'].unique())]
pozos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27727 entries, 0 to 84330
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               27727 non-null  int64  
 1   sigla                    27727 non-null  object 
 2   idpozo                   27727 non-null  int64  
 3   area                     27727 non-null  object 
 4   cod_area                 27727 non-null  object 
 5   empresa                  27727 non-null  object 
 6   yacimiento               27727 non-null  object 
 7   cod_yacimiento           27727 non-null  object 
 8   formacion                27699 non-null  object 
 9   cuenca                   27727 non-null  object 
 10  provincia                27727 non-null  object 
 11  cota                     27727 non-null  float64
 12  profundidad              27727 non-null  float64
 13  clasificacion            27727 non-null  object 
 14  subclasificacion         27

# Unicidad sigla-formacion

Veo duplicados en sigla y formacion. Un duplicado difiere en la provincia (Santa Cruz / Tierra del fuego), o difiere en la empresa. Las producciones son practicamente las mismas, muy parecidas, podemos decir que los duplicados son el mismo pozo.

In [38]:
duplicados = pozos[pozos[['sigla','formacion']].duplicated(keep=False)]
duplicados.to_csv("duplicados_sigla_form.csv")
duplicados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 3381 to 60478
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               80 non-null     int64  
 1   sigla                    80 non-null     object 
 2   idpozo                   80 non-null     int64  
 3   area                     80 non-null     object 
 4   cod_area                 80 non-null     object 
 5   empresa                  80 non-null     object 
 6   yacimiento               80 non-null     object 
 7   cod_yacimiento           80 non-null     object 
 8   formacion                80 non-null     object 
 9   cuenca                   80 non-null     object 
 10  provincia                80 non-null     object 
 11  cota                     80 non-null     float64
 12  profundidad              80 non-null     float64
 13  clasificacion            80 non-null     object 
 14  subclasificacion         80

In [39]:
def merge_provincias_api (pozos):
    siglas = pozos['sigla'].unique()
    for sigla in siglas:
        pozos_sigla = pozos[pozos['sigla'] == sigla]
        provi_dist = len(pozos_sigla['provincia'].unique())
        if (provi_dist > 1):
            ids = pozos_sigla['idpozo']
            for i in (pozos[pozos['idpozo'].isin(ids)].index):
                pozos.loc[i, 'provincia'] = pozos_sigla['provincia_nombre'].iloc[0]

In [40]:
merge_provincias_api(pozos)

# Unicidad de sigla

Me fijo de los que tienen distintas coordenadas, cuales tienen misma boca. Esperaríamos que no haya ninguna por lo mismo de antes, si no hay coordenadas repetidas, en principio no debería haber dos siglas iguales. Sin embargo hay 294

In [54]:
sigla_dup = pozos[(pozos['sigla'].duplicated(keep=False))]
sigla_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2550 entries, 224 to 84301
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2550 non-null   int64  
 1   sigla                    2550 non-null   object 
 2   idpozo                   2550 non-null   int64  
 3   area                     2550 non-null   object 
 4   cod_area                 2550 non-null   object 
 5   empresa                  2550 non-null   object 
 6   yacimiento               2550 non-null   object 
 7   cod_yacimiento           2550 non-null   object 
 8   formacion                2550 non-null   object 
 9   cuenca                   2550 non-null   object 
 10  provincia                2550 non-null   object 
 11  cota                     2550 non-null   float64
 12  profundidad              2550 non-null   float64
 13  clasificacion            2550 non-null   object 
 14  subclasificacion         2

In [55]:
def coords_notclose (df):
    siglas = df['sigla'].unique()
    ids = []
    
    for sigla in siglas:
        pozos_eqsigla = df[df['sigla'] == sigla]
        lon_close = np.isclose(pozos_eqsigla['lon'], pozos_eqsigla['lon'].iloc[0], atol=1e-3)
        lat_close = np.isclose(pozos_eqsigla['lat'], pozos_eqsigla['lat'].iloc[0], atol=1e-3)
        
        for p in range (len(lon_close)):
            if not (lon_close[p] & lat_close[p]):
                for i in range (len(lon_close)):
                    ids.append(pozos_eqsigla['idpozo'].iloc[i])
                break
                
    return ids

dup_sigla_notclosecoord = sigla_dup[sigla_dup['idpozo'].isin(coords_notclose(sigla_dup))]
dup_sigla_notclosecoord.to_csv("duplicados_sigla_notclosecoord.csv")
dup_sigla_notclosecoord.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               0 non-null      int64  
 1   sigla                    0 non-null      object 
 2   idpozo                   0 non-null      int64  
 3   area                     0 non-null      object 
 4   cod_area                 0 non-null      object 
 5   empresa                  0 non-null      object 
 6   yacimiento               0 non-null      object 
 7   cod_yacimiento           0 non-null      object 
 8   formacion                0 non-null      object 
 9   cuenca                   0 non-null      object 
 10  provincia                0 non-null      object 
 11  cota                     0 non-null      float64
 12  profundidad              0 non-null      float64
 13  clasificacion            0 non-null      object 
 14  subclasificacion         0 non-null      ob

In [50]:
import pandas as pd
import json
def make_coordinates_unique (pozos, duplicados):
    siglas = duplicados['sigla'].unique()
    for sigla in siglas:
        pozos_sigla = pozos[pozos['sigla'] == sigla]
        ids = pozos_sigla['idpozo']
        geojson = pozos_sigla['geojson'].iloc[0]
        geom = pozos_sigla['geom'].iloc[0]
        for i in (pozos[pozos['idpozo'].isin(ids)].index):                  
            pozos.loc[i, 'geojson'] = geojson
            pozos.loc[i, 'geom'] = geom
            
def recalculate_coordinates(df):
    # Eliminar columnas específicas si existen
    df.drop(columns=['coordinates', 'coordinates_str', 'lon', 'lat'], axis=1, errors='ignore', inplace=True)
    
    # Crear una columna temporal para procesar el contenido sin alterar 'geojson'
    df['geojson_dic'] = df['geojson'].apply(lambda x: json.loads(x.replace("'", '"')))
    
    # Extraer las coordenadas en nuevas columnas 'lon' y 'lat'
    df[['lon', 'lat']] = pd.DataFrame(
        df['geojson_dic'].apply(lambda x: x['coordinates']).to_list(), 
        index=df.index
    )
    
    # Eliminar la columna temporal para mantener limpio el DataFrame
    df.drop(columns=['geojson_dic'], errors='ignore', inplace=True)
   

A los que tienen distinta coordenada, los unifico (su geojson) y luego recalculo las coordenadas para tenerlas bien

In [53]:
make_coordinates_unique(pozos, dup_sigla_notclosecoord)
recalculate_coordinates(pozos)

# Unicidad de coordenadas

Me fijo geoms repetidos

In [13]:
geom_dup = pozos[pozos['geom'].duplicated(keep=False)]
geom_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2387 entries, 239 to 84298
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2387 non-null   int64  
 1   sigla                    2387 non-null   object 
 2   idpozo                   2387 non-null   int64  
 3   area                     2387 non-null   object 
 4   cod_area                 2387 non-null   object 
 5   empresa                  2387 non-null   object 
 6   yacimiento               2387 non-null   object 
 7   cod_yacimiento           2387 non-null   object 
 8   formacion                2387 non-null   object 
 9   cuenca                   2387 non-null   object 
 10  provincia                2387 non-null   object 
 11  cota                     2387 non-null   float64
 12  profundidad              2387 non-null   float64
 13  clasificacion            2387 non-null   object 
 14  subclasificacion         2

Me fijo geojsons repetidos

In [14]:
geojson_dup = pozos[pozos['geojson'].duplicated(keep=False)]
geojson_dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2387 entries, 239 to 84298
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2387 non-null   int64  
 1   sigla                    2387 non-null   object 
 2   idpozo                   2387 non-null   int64  
 3   area                     2387 non-null   object 
 4   cod_area                 2387 non-null   object 
 5   empresa                  2387 non-null   object 
 6   yacimiento               2387 non-null   object 
 7   cod_yacimiento           2387 non-null   object 
 8   formacion                2387 non-null   object 
 9   cuenca                   2387 non-null   object 
 10  provincia                2387 non-null   object 
 11  cota                     2387 non-null   float64
 12  profundidad              2387 non-null   float64
 13  clasificacion            2387 non-null   object 
 14  subclasificacion         2

In [15]:
# Checkeo geom = geojson
False in (geom_dup.index == geojson_dup.index)

False

In [16]:
# Cantidad de coordenadas unicas
len(geom_dup['geom'].unique())

1092

### Ejemplo duplicado

In [22]:
geom_dup[(geom_dup['geom'] == geom_dup['geom'].iloc[500])]

,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,provincia,...,tipoextraccion,tipoestado,adjiv_fecha_inicio_perf,adjiv_fecha_fin_perf,adjiv_fecha_inicio_term,adjiv_fecha_fin_term,geojson,geom,lon,lat
42188,YPF.Nq.EL-22,109903,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,rayoso,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2004-07-02,2004-07-15,2004-07-29,2004-08-14,"{""type"":""Point"",""coordinates"":[-69.2757194,-37...",0101000020E6100000DB77FB62A55151C0F5A9190838AE...,-69.275719,-37.361085
42189,YPF.Nq.EL-22,109904,CHIHUIDO DE LA SIERRA NEGRA,CSN,YPF S.A.,EL LIMITE,Z135,agrio,NEUQUINA,Neuquén,...,Bombeo Mecánico,Extracción Efectiva,2004-07-02,2004-07-15,2004-07-29,2004-08-14,"{""type"":""Point"",""coordinates"":[-69.2757194,-37...",0101000020E6100000DB77FB62A55151C0F5A9190838AE...,-69.275719,-37.361085


In [23]:
sample_dup = produccion[(produccion['idpozo'] == 109903) | (produccion['idpozo'] == 109904)]
sample_dup[(sample_dup['prod_pet'] != 0) & (sample_dup['prod_gas'] != 0)]

,idempresa,anio,mes,idpozo,prod_pet,prod_gas,prod_agua,iny_agua,iny_gas,iny_co2,...,areayacimiento,cuenca,provincia,tipo_de_recurso,proyecto,clasificacion,subclasificacion,sub_tipo_recurso,fecha_data,index
36661,YPF,2024,1,109904,15.78,0.07,161.95,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36661
36662,YPF,2024,1,109903,36.83,0.15,377.89,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-01-31,36662
75000,YPF,2024,2,109904,13.38,0.06,157.97,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75000
75001,YPF,2024,2,109903,31.22,0.15,368.58,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-02-29,75001
112625,YPF,2024,3,109904,15.13,0.07,172.38,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112625
112626,YPF,2024,3,109903,35.30,0.16,402.23,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-03-31,112626
154639,YPF,2024,4,109904,15.16,0.07,174.34,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154639
154640,YPF,2024,4,109903,35.38,0.16,406.80,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-04-30,154640
190633,YPF,2024,5,109904,12.93,0.06,153.19,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190633
190634,YPF,2024,5,109903,30.18,0.14,357.43,0.0,0.0,0.0,...,EL LIMITE,NEUQUINA,Neuquén,CONVENCIONAL,Sin Proyecto,EXPLOTACION,DESARROLLO,NaN,2024-05-31,190634


In [14]:
geom_dup.loc[42188] == geom_dup.loc[42189]

sigla                       True
idpozo                     False
area                        True
cod_area                    True
empresa                     True
yacimiento                  True
cod_yacimiento              True
formacion                  False
cuenca                      True
provincia                   True
cota                        True
profundidad                 True
clasificacion               True
subclasificacion            True
tipo_recurso                True
sub_tipo_recurso            True
gasplus                     True
tipopozo                    True
tipoextraccion              True
tipoestado                  True
adjiv_fecha_inicio_perf     True
adjiv_fecha_fin_perf        True
adjiv_fecha_inicio_term     True
adjiv_fecha_fin_term        True
geojson                     True
geom                        True
lon                        False
lat                        False
dtype: bool

### Checkeo cuáles tienen bocas unicas

Me fijo cuáles tienen bocas unicas dentro de mismas coordenadas. Esperaríamos que no haya ninguna ya que una boca es un punto fisico en un terreno, por lo que no debería existir una boca unica en una lista de coordenadas repetidas. Sin embargo hay 155

In [17]:
geom_dup_uneqbocas = geom_dup[~geom_dup['sigla'].duplicated(keep=False)]
geom_dup_uneqbocas.to_csv("duplicados_eqcoord_uneqsigla.csv")
geom_dup_uneqbocas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 655 to 83928
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               155 non-null    int64  
 1   sigla                    155 non-null    object 
 2   idpozo                   155 non-null    int64  
 3   area                     155 non-null    object 
 4   cod_area                 155 non-null    object 
 5   empresa                  155 non-null    object 
 6   yacimiento               155 non-null    object 
 7   cod_yacimiento           155 non-null    object 
 8   formacion                155 non-null    object 
 9   cuenca                   155 non-null    object 
 10  provincia                155 non-null    object 
 11  cota                     155 non-null    float64
 12  profundidad              155 non-null    float64
 13  clasificacion            155 non-null    object 
 14  subclasificacion         15

Un ejemplo, difiere solo en formacion y sigla, es posible que hayan anotado diferente al anotar distintas formaciones

In [18]:
geom_dup[geom_dup['geom'] == geom_dup_uneqbocas['geom'].iloc[150]]

,Unnamed: 0,sigla,idpozo,area,cod_area,empresa,yacimiento,cod_yacimiento,formacion,cuenca,...,coordinates_str,info_coordenadas,departamento_id,departamento_nombre,lat,lon,municipio_id,municipio_nombre,provincia_id,provincia_nombre
83585,83585,SHE.Nq.CASO-16(h),164930,COIRON AMARGO SUR OESTE,CAOS,SHELL ARGENTINA S.A.,COIRON AMARGO SUR OESTE,CASO,vaca muerta,NEUQUINA,...,"-68.496745133684, -38.3870095874503","{'departamento_id': '58014', 'departamento_nom...",58014.0,Añelo,-38.38701,-68.496745,NaN,NaN,58.0,Neuquén
83589,83589,SHE.Nq.CASO-16(h)v,164934,CRUZ DE LORENA,CDLO,SHELL ARGENTINA S.A.,CRUZ DE LORENA,CDLO,vaca muerta,NEUQUINA,...,"-68.496745133684, -38.3870095874503","{'departamento_id': '58014', 'departamento_nom...",58014.0,Añelo,-38.38701,-68.496745,NaN,NaN,58.0,Neuquén
